In [1]:
#Import file & resources

import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date

In [2]:
#Read file into df & preview loaded data

df = pd.read_csv("Android apps.csv", low_memory=False)

df


,appname,fancyname,category,company,extrainfo,source,purchases,ads,sharesinfo,shareslocation,...,price,secondprice,paidapp,rating,numberreviews,five,four,three,two,one
0,com.netflix.mediaclient,Netflix,entertainment,"netflix, inc.","offers in-app purchases \ august 19, 2019 \ va...",google play,1.0,0.0,0.0,0.0,...,free,NaN,0.0,4.5,"7,287,852",100,13,4,2,9
1,com.facebook.orca,Messenger ? Text and Video Chat for Free,communication,facebook,"offers in-app purchases \ august 20, 2019 \ va...",google play,1.0,0.0,0.0,1.0,...,free,NaN,0.0,4.2,"69,050,158",100,15,9,4,15
2,com.android.chrome,Google Chrome: Fast & Secure,communication,google llc,"august 14, 2019 \ varies with device \ 5,000,0...",google play,0.0,0.0,0.0,0.0,...,free,NaN,0.0,4.3,"17,065,648",100,18,9,4,10
3,com.google.android.gm,Gmail,communication,google llc,"contains ads \ august 19, 2019 \ varies with d...",google play,0.0,1.0,0.0,0.0,...,free,NaN,0.0,4.4,"6,272,191",100,16,7,3,9
4,com.grindrapp.android,Grindr - Gay chat,social,grindr llc,contains ads?offers in-app purchases \ august ...,google play,1.0,1.0,0.0,0.0,...,free,NaN,0.0,3.5,"365,432",100,21,17,12,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50212,mobi.easyorderapp.slagerijthyssen,EasyOrder,shopping,easyorder,shopping| | |,apkpure,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50213,com.thalys.selfservice,e-PRESS&MORE by Thalys,news & magazines,thalys,"june 28, 2019 \ 48m \ 1,000+ \ 3.1.1 \ 4.4 and...",google play,1.0,0.0,0.0,0.0,...,free,NaN,0.0,2.2,26,37,0,12,12,100
50214,com.freeme.camera,Dreame,books & reference,dreame media,"offers in-app purchases \ october 16, 2019 \ 1...",google play,1.0,0.0,0.0,0.0,...,free,NaN,0.0,4.5,"103,31",100,24,12,2,2
50215,com.thalys.missionreport,e-PRESS&MORE by Thalys,news & magazines,thalys,"june 28, 2019 \ 48m \ 1,000+ \ 3.1.1 \ 4.4 and...",google play,1.0,0.0,0.0,0.0,...,free,NaN,0.0,2.2,26,37,0,12,12,100


In [3]:
# List available columns

df.columns

Index(['appname', 'fancyname', 'category', 'company', 'extrainfo', 'source',
       'purchases', 'ads', 'sharesinfo', 'shareslocation',
       'unrestrictedinternet', 'usersinteract', 'game', 'age_rating',
       'Parentalguidance', 'Downloads', 'FuzzyMatched', 'basename',
       'SystemApp', 'New.category', 'categorygame', 'low_price_item',
       'top_price_item', 'price', 'secondprice', 'paidapp', 'rating',
       'numberreviews', 'five', 'four', 'three', 'two', 'one'],
      dtype='object')

In [4]:
#Filter out non-game app data
print(f"size before filter: {len(df)}")
df = df.loc[df['game']==1]
print(f"size after filtering by game: {len(df)}")

#Filter out invalid data
df = df.loc[(df['numberreviews'] != 'error during scraping') & (df['numberreviews'] != 'rating disabled') & (df['numberreviews'] != 'not found in databases')]
print(f"size after filtering out invalid numberreviews values: {len(df)}")
#df

size before filter: 50217
size after filtering by game: 11109
size after filtering out invalid numberreviews values: 10621


In [5]:
#Keep features/columns

columns_keep = 'appname	fancyname	company	purchases	ads	sharesinfo	shareslocation	unrestrictedinternet	usersinteract	game	age_rating	Parentalguidance	Downloads	category	low_price_item	top_price_item	price	paidapp	rating	numberreviews'.split('\t')

df = df[columns_keep]
#df

In [6]:
# Drop nan of numberreviews

df_notna = df[df['numberreviews'].notna()]
#df_notna

In [7]:
# Create function to replace specific string characters in numberreviews for conversion purposes

def CleanNumberString(numberString):
    #print(f"number to cleanse: {numberString}")
    cleanedString = numberString.replace(",","").replace("","").replace("k","").replace("+","").replace("m","")
    #print (f"cleaned string: {cleanedString}")
    return cleanedString

In [8]:
# Create function chained with CleanNumberString to convert numerical objects to numerical values

def CleanNumber(numberString):
    multiplier = 1

    if('k' in numberString):
        multiplier = 1000
    elif('m' in numberString):
        multiplier = 1000000
        
    cleanedString = float(CleanNumberString(numberString)) * multiplier    
    
    return cleanedString

In [9]:
# Create clean numerical values in string format in numberreviews column

df_notna["cleaned_numberReviews"] = df_notna["numberreviews"]
#df_notna
df_notna['cleaned_numberReviews'] = df_notna.apply(lambda row: CleanNumber(row['cleaned_numberReviews']), axis=1)

C:\Users\artem\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\artem\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [10]:
# Double check whether function is working as planned

df_notna[df_notna['numberreviews'].str.contains("m")]

,appname,fancyname,company,purchases,ads,sharesinfo,shareslocation,unrestrictedinternet,usersinteract,game,...,Parentalguidance,Downloads,category,low_price_item,top_price_item,price,paidapp,rating,numberreviews,cleaned_numberReviews
41728,com.gameloft.android.ANMP.GloftSIHM,MARVEL Spider-Man Unlimited,gameloft,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0.0,NaN,action,NaN,NaN,free,0.0,4.2,2.4m+,2400000.0
41771,com.gameloft.android.ANMP.GloftUOHM,UNO ? & Friends,gameloft,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0.0,NaN,card,NaN,NaN,free,0.0,4.1,1.69m+,1690000.0
42130,com.disney.WMWLite,Where's My Water? Free,disney,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0.0,NaN,puzzle,NaN,NaN,free,0.0,4.3,1.4m+,1400000.0
42131,com.pikpok.turbo,Turbo FAST,pikpok,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0.0,NaN,racing,NaN,NaN,free,0.0,4.3,1.34m+,1340000.0
42311,com.umonistudio.tile,Don't Tap The White Tile,cheetah games,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0.0,NaN,arcade,NaN,NaN,free,0.0,4.3,2.96m+,2960000.0
42594,com.nordcurrent.Games101,101-in-1 Games,nordcurrent,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0.0,NaN,arcade,NaN,NaN,free,0.0,4.4,1.1m+,1100000.0
42795,com.izmo.onlinekafatopu,Online Head Ball 3,masomo gaming,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0.0,NaN,sports,NaN,NaN,free,0.0,4.4,1.13m+,1130000.0
44077,mobi.MultiCraft,<U+25BA> MultiCraft <U+2015> Free Miner! <U+00...,multicraft official,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0.0,NaN,adventure,NaN,NaN,free,0.0,4.3,1.32m+,1320000.0
44209,com.activision.callofduty.heroes,Call of Duty?: Heroes,"activision publishing, inc.",NaN,NaN,NaN,NaN,NaN,NaN,1,...,0.0,NaN,action,NaN,NaN,free,0.0,4.4,1.61m+,1610000.0
47013,com.telltalegames.minecraft100,Minecraft: Story Mode,telltale games,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0.0,NaN,adventure,NaN,NaN,free,0.0,4.3,1.11m+,1110000.0


In [11]:
# Attempt to cast cleansed column as int - should throw errors if cleansing is incomplete
df_notna.cleaned_numberReviews = df_notna.cleaned_numberReviews.astype(np.int64)
df_notna

C:\Users\artem\.conda\envs\PythonData\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,appname,fancyname,company,purchases,ads,sharesinfo,shareslocation,unrestrictedinternet,usersinteract,game,...,Parentalguidance,Downloads,category,low_price_item,top_price_item,price,paidapp,rating,numberreviews,cleaned_numberReviews
228,com.gamebasics.osm,Online Soccer Manager (OSM),gamebasics bv,1.0,1.0,0.0,0.0,0.0,1.0,1,...,0.0,10000000.0,sports,0.59,109.99,free,0.0,4.5,"1,656,120",1656120
235,com.fgol.HungrySharkEvolution,Hungry Shark Evolution,ubisoft entertainment,1.0,1.0,0.0,0.0,0.0,0.0,1,...,0.0,100000000.0,arcade,2.29,109.99,free,0.0,4.5,"6,495,234",6495234
246,com.zynga.livepoker,Zynga Poker ? Free Texas Holdem Online Card Games,zynga,1.0,1.0,1.0,0.0,0.0,1.0,1,...,0.0,50000000.0,casino,0.59,350.00,free,0.0,4.5,"2,315,552",2315552
247,ee.dustland.android.dustlandsudoku,Sudoku - The Clean One,dustland design,1.0,1.0,0.0,0.0,0.0,0.0,1,...,0.0,1000000.0,puzzle,1.89,2.99,free,0.0,4.6,"6,639",6639
254,com.miniclip.eightballpool,8 Ball Pool,miniclip.com,1.0,1.0,0.0,0.0,0.0,0.0,1,...,0.0,500000000.0,sports,0.79,99.99,free,0.0,4.5,"17,053,699",17053699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50172,com.ea.tetrisfree_row,TETRIS Blitz,electronic arts,1.0,0.0,0.0,0.0,0.0,0.0,1,...,0.0,10000000.0,puzzle,1.09,109.99,free,0.0,4.2,"250,381",250381
50188,com.worms2armageddon.apptnb,Worms 2: Armageddon,team 17 digital limited,0.0,0.0,0.0,0.0,0.0,0.0,1,...,0.0,1000000.0,strategy,NaN,NaN,4.49,1.0,3.9,"61,603",61603
50190,com.nianticlabs.hpwu.prod.ares,Harry Potter: Wizards Unite,"niantic, inc.",1.0,0.0,0.0,0.0,0.0,0.0,1,...,0.0,5000000.0,adventure,1.09,109.99,free,0.0,3.9,"268,773",268773
50192,com.daysofwonder.smallworld2_humble,Small World 2,no info,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0.0,NaN,board,NaN,NaN,free,0.0,4.3,5.66k+,5660


In [12]:
# Get distribution of number of reviews values

df_notna['cleaned_numberReviews'].describe()


count    1.052500e+04
mean     1.564649e+05
std      1.021994e+06
min      1.000000e+00
25%      1.010000e+03
50%      8.789000e+03
75%      5.307300e+04
max      5.020212e+07
Name: cleaned_numberReviews, dtype: float64

In [13]:
# Keep only rows that have at least 1,000 reviews

df_notna = df_notna.loc[df_notna['cleaned_numberReviews'] >= 1000]

In [14]:
# Double check ratings data for any quirks

df_notna.rating.unique()

array(['4.5', '4.6', '3.4', '4.3', '4.2', '4.4', '3.8', '4.0', '4.7',
       '3.2', '3.5', '4.1', '3.7', '3.9', '3.6', '3.3', '4.8', '3.1',
       '3.0', '2.5', '2.9', '2.7', '2.6', '2.8', '2.3', '2.4', '2.2',
       '4.9', '1.7', '2.0', '1.9', '1.5', '2.1', '1.8', '1.3', '1.6'],
      dtype=object)

In [15]:
# Cast ratings as float type

df_notna.rating = df_notna.rating.astype(np.float)

C:\Users\artem\.conda\envs\PythonData\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [16]:
# Double check latest dtypes on df

df_notna.dtypes

appname                   object
fancyname                 object
company                   object
purchases                float64
ads                      float64
sharesinfo               float64
shareslocation           float64
unrestrictedinternet     float64
usersinteract            float64
game                       int64
age_rating                object
Parentalguidance         float64
Downloads                float64
category                  object
low_price_item            object
top_price_item            object
price                     object
paidapp                  float64
rating                   float64
numberreviews             object
cleaned_numberReviews      int64
dtype: object

In [17]:
# Load Google play data

df_gp = pd.read_csv("Google-Playstore.csv", low_memory=False)

In [18]:
# Preview Google play data

df_gp

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice
0,HTTrack Website Copier,com.httrack.android,Communication,3.6,2848.0,"100,000+",100000.0,351560,True,0.0,...,Xavier Roche,http://www.httrack.com/,roche+android@httrack.com,"Aug 12, 2013","May 20, 2017",Everyone,http://android.httrack.com/privacy-policy.html,False,False,False
1,World War 2: Offline Strategy,com.skizze.wwii,Strategy,4.3,17297.0,"1,000,000+",1000000.0,2161778,True,0.0,...,Skizze Games,http://stereo7.com/,Skizze.Games@gmail.com,"Jul 19, 2018","Nov 26, 2020",Everyone 10+,https://www.iubenda.com/privacy-policy/8032781,True,True,False
2,WPSApp,com.themausoft.wpsapp,Tools,4.2,488639.0,"50,000,000+",50000000.0,79304739,True,0.0,...,TheMauSoft,http://www.themausoft.com,wpsapp.app@gmail.com,"Mar 7, 2016","Oct 21, 2020",Everyone,https://sites.google.com/view/wpsapppolicy/main,True,False,False
3,"OfficeSuite - Office, PDF, Word, Excel, PowerP...",com.mobisystems.office,Business,4.2,1224420.0,"100,000,000+",100000000.0,163660067,True,0.0,...,MobiSystems,http://www.mobisystems.com,support-officesuite-android@mobisystems.com,"Dec 22, 2011","Nov 23, 2020",Everyone,http://www.mobisystems.com/mobile/privacy-poli...,True,True,False
4,Loud Player Free,com.arthelion.loudplayer,Music & Audio,4.2,665.0,"50,000+",50000.0,73463,True,0.0,...,Arthelion92,http://www.arthelion.com,arthelion92@gmail.com,"Sep 24, 2016","Nov 22, 2020",Everyone,http://www.arthelion.com/index.php/fr/android-...,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118131,FleetEnable,com.fleetenable.app,Productivity,NaN,NaN,10+,10.0,10,True,0.0,...,Imaginnovate,http://fleetenable.com/,krishna@imaginnovate.com,NaN,"Jun 30, 2020",Everyone,http://fleetenable.com/,False,False,False
1118132,AutoThink,com.fleetenable.autothink,Productivity,0.0,0.0,100+,100.0,127,True,0.0,...,Imaginnovate,http://www.imaginnovate.com,krishna@imaginnovate.com,"Nov 26, 2019","Nov 13, 2020",Everyone,http://www.imaginnovate.com,False,False,False
1118133,FieldEnable,com.fieldenable.app.fieldenable,Business,0.0,0.0,100+,100.0,282,True,0.0,...,Imaginnovate,http://www.imaginnovate.com,FieldEnable@imaginnovate.com,"Apr 30, 2018","Nov 30, 2020",Everyone,http://tw.fieldenable.com/privacy,False,False,False
1118134,Live Concert,com.varbin.liveconcert,Events,0.0,0.0,500+,500.0,986,True,0.0,...,Varbin Softwares,http://varbin.com,varbinsoftware@gmail.com,"Dec 20, 2017","Dec 20, 2017",Everyone,http://varbin.com/live-concert-privacy-policy,False,False,False


In [19]:
# Merge Google play data with Android apps data. Join is preformed on the app name

df_merged = pd.merge(df_notna, df_gp, how='inner', left_on='appname', right_on='App Id')

In [20]:
# Drop duplicate key column

df_merged = df_merged.drop("App Id", axis=1)

In [21]:
# Preview merged dataset

df_merged

,appname,fancyname,company,purchases,ads,sharesinfo,shareslocation,unrestrictedinternet,usersinteract,game,...,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice
0,com.gamebasics.osm,Online Soccer Manager (OSM),gamebasics bv,1.0,1.0,0.0,0.0,0.0,1.0,1,...,Gamebasics BV,http://www.onlinesoccermanager.com,en.support@onlinesoccermanager.com,"Oct 4, 2012","Nov 26, 2020",Everyone,http://www.onlinesoccermanager.com/Privacy,True,True,False
1,com.fgol.HungrySharkEvolution,Hungry Shark Evolution,ubisoft entertainment,1.0,1.0,0.0,0.0,0.0,0.0,1,...,Ubisoft Entertainment,https://www.ubisoft.com/,android.support@ubisoft.com,"Feb 22, 2013","Oct 27, 2020",Teen,http://r-mob.ubi.com/?a=PrivacyPolicyUS,True,True,True
2,com.zynga.livepoker,Zynga Poker ? Free Texas Holdem Online Card Games,zynga,1.0,1.0,1.0,0.0,0.0,1.0,1,...,Zynga,https://www.zynga.com/,android.support@zynga.com,"Dec 21, 2010","Nov 04, 2020",Teen,https://www.zynga.com/privacy/policy,True,True,False
3,ee.dustland.android.dustlandsudoku,Sudoku - The Clean One,dustland design,1.0,1.0,0.0,0.0,0.0,0.0,1,...,Dustland Design,http://dustland.ee,sudoku@dustland.ee,"Jul 20, 2017","Oct 29, 2020",Everyone,http://dustland.ee/sudoku/privacy-policy/,True,True,False
4,com.miniclip.eightballpool,8 Ball Pool,miniclip.com,1.0,1.0,0.0,0.0,0.0,0.0,1,...,Miniclip.com,https://support.miniclip.com/hc/categories/115...,support@miniclip.com,"Jan 23, 2013","Nov 26, 2020",Everyone,http://www.miniclip.com/android/privacy-policy/,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6037,com.berniiiiiiii.logomatchup,"""Memory"" - Memory game",berni mobile,NaN,NaN,NaN,NaN,NaN,NaN,1,...,Berni Mobile,http://www.bernimobile.com/,girblumk@gmail.com,"Oct 26, 2012","Nov 28, 2020",Everyone,http://www.bernimobile.com/PrivatePolicy,True,False,False
6038,com.gipnetix.berryking,Berry King,gameborn inc.,NaN,NaN,NaN,NaN,NaN,NaN,1,...,GAMEBORN Inc.,https://gameborn.io,support@gameborn.io,"Jul 11, 2014","Aug 20, 2020",Everyone,https://gameborn.io/privacy-policy,True,True,False
6039,com.smilerlee.jewels,Jewels Switch,words mobile,NaN,NaN,NaN,NaN,NaN,NaN,1,...,Words Mobile,http://www2.words-mobile.com,contact@words-mobile.com,"Jun 13, 2013","Dec 26, 2019",Everyone,http://www2.words-mobile.com/policy.htm,True,True,False
6040,com.geargames.pfp,Prehistoric Park Builder,gear games,NaN,NaN,NaN,NaN,NaN,NaN,1,...,Gear Games,http://prehistoric-games.com/,info@gear-games.com,"Nov 28, 2012","Feb 17, 2014",Everyone,http://gear-games.com/privacy/,True,True,False


In [22]:
# Create the Companies dataframe, to be used for determining whether a company is experienced or not.
# A simple row count is used after grouping the data by company in order to derive the number of apps the company has produced.

df_companies = df_merged.groupby("company")["company"].count().reset_index(name="gamesCount")
df_companies

,company,gamesCount
0,(<u+c8fc>)<u+c5d4><u+be44><u+c83c><u+d06c><u+b...,1
1,(andrei & aleksandr krupiankou),1
2,0.1%,2
3,0km apps,1
4,100500games.org,1
...,...,...
3142,zut!,1
3143,zuuks games,3
3144,zy puzzle games,1
3145,zynga,19


In [23]:
# Convert the Companies dataframe to list of dicts, in order to be used in the UDF below

companiesList = df_companies.to_dict('records')
companiesList

[{'company': '(<u+c8fc>)<u+c5d4><u+be44><u+c83c><u+d06c><u+b9ac><u+c5d0><u+c774><u+d2f0><u+be0c>',
  'gamesCount': 1},
 {'company': '(andrei & aleksandr krupiankou)', 'gamesCount': 1},
 {'company': '0.1%', 'gamesCount': 2},
 {'company': '0km apps', 'gamesCount': 1},
 {'company': '100500games.org', 'gamesCount': 1},
 {'company': '101xp limited', 'gamesCount': 1},
 {'company': '10p studio', 'gamesCount': 2},
 {'company': '11 bit studios', 'gamesCount': 2},
 {'company': '111%', 'gamesCount': 4},
 {'company': '137studio', 'gamesCount': 1},
 {'company': '1492 studio', 'gamesCount': 9},
 {'company': '17studio', 'gamesCount': 1},
 {'company': '17th pixel', 'gamesCount': 1},
 {'company': '17th pixel poland', 'gamesCount': 1},
 {'company': '1905 games', 'gamesCount': 1},
 {'company': '1bsyl', 'gamesCount': 3},
 {'company': '1coin', 'gamesCount': 1},
 {'company': '1der entertainment', 'gamesCount': 1},
 {'company': '1games', 'gamesCount': 1},
 {'company': '1gravity llc', 'gamesCount': 1},
 {'com

In [24]:
# Returns True if the given company has a game count over 1

def IsCompanyExperienced(companiesList, company):
    companyGamesCount = next((listCompany["gamesCount"] for listCompany in companiesList if company == listCompany["company"]), None)
    if(companyGamesCount > 1):
        return True
    else:
        return False

In [25]:
# Runs the above UDF to derive the CompanyIsExperienced column

df_merged["CompanyIsExperienced"] = df_merged.apply(lambda row: IsCompanyExperienced(companiesList, row["company"]),axis=1)

In [26]:
# Preview the data, complete with the CompanyIsExperienced column

df_merged

,appname,fancyname,company,purchases,ads,sharesinfo,shareslocation,unrestrictedinternet,usersinteract,game,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,CompanyIsExperienced
0,com.gamebasics.osm,Online Soccer Manager (OSM),gamebasics bv,1.0,1.0,0.0,0.0,0.0,1.0,1,...,http://www.onlinesoccermanager.com,en.support@onlinesoccermanager.com,"Oct 4, 2012","Nov 26, 2020",Everyone,http://www.onlinesoccermanager.com/Privacy,True,True,False,False
1,com.fgol.HungrySharkEvolution,Hungry Shark Evolution,ubisoft entertainment,1.0,1.0,0.0,0.0,0.0,0.0,1,...,https://www.ubisoft.com/,android.support@ubisoft.com,"Feb 22, 2013","Oct 27, 2020",Teen,http://r-mob.ubi.com/?a=PrivacyPolicyUS,True,True,True,True
2,com.zynga.livepoker,Zynga Poker ? Free Texas Holdem Online Card Games,zynga,1.0,1.0,1.0,0.0,0.0,1.0,1,...,https://www.zynga.com/,android.support@zynga.com,"Dec 21, 2010","Nov 04, 2020",Teen,https://www.zynga.com/privacy/policy,True,True,False,True
3,ee.dustland.android.dustlandsudoku,Sudoku - The Clean One,dustland design,1.0,1.0,0.0,0.0,0.0,0.0,1,...,http://dustland.ee,sudoku@dustland.ee,"Jul 20, 2017","Oct 29, 2020",Everyone,http://dustland.ee/sudoku/privacy-policy/,True,True,False,False
4,com.miniclip.eightballpool,8 Ball Pool,miniclip.com,1.0,1.0,0.0,0.0,0.0,0.0,1,...,https://support.miniclip.com/hc/categories/115...,support@miniclip.com,"Jan 23, 2013","Nov 26, 2020",Everyone,http://www.miniclip.com/android/privacy-policy/,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6037,com.berniiiiiiii.logomatchup,"""Memory"" - Memory game",berni mobile,NaN,NaN,NaN,NaN,NaN,NaN,1,...,http://www.bernimobile.com/,girblumk@gmail.com,"Oct 26, 2012","Nov 28, 2020",Everyone,http://www.bernimobile.com/PrivatePolicy,True,False,False,True
6038,com.gipnetix.berryking,Berry King,gameborn inc.,NaN,NaN,NaN,NaN,NaN,NaN,1,...,https://gameborn.io,support@gameborn.io,"Jul 11, 2014","Aug 20, 2020",Everyone,https://gameborn.io/privacy-policy,True,True,False,False
6039,com.smilerlee.jewels,Jewels Switch,words mobile,NaN,NaN,NaN,NaN,NaN,NaN,1,...,http://www2.words-mobile.com,contact@words-mobile.com,"Jun 13, 2013","Dec 26, 2019",Everyone,http://www2.words-mobile.com/policy.htm,True,True,False,True
6040,com.geargames.pfp,Prehistoric Park Builder,gear games,NaN,NaN,NaN,NaN,NaN,NaN,1,...,http://prehistoric-games.com/,info@gear-games.com,"Nov 28, 2012","Feb 17, 2014",Everyone,http://gear-games.com/privacy/,True,True,False,True


In [28]:
# Check data types, row counts
df_merged.dtypes

appname                   object
fancyname                 object
company                   object
purchases                float64
ads                      float64
sharesinfo               float64
shareslocation           float64
unrestrictedinternet     float64
usersinteract            float64
game                       int64
age_rating                object
Parentalguidance         float64
Downloads                float64
category                  object
low_price_item            object
top_price_item            object
price                     object
paidapp                  float64
rating                   float64
numberreviews             object
cleaned_numberReviews      int64
App Name                  object
Category                  object
Rating                   float64
Rating Count             float64
Installs                  object
Minimum Installs         float64
Maximum Installs           int64
Free                        bool
Price                    float64
Currency  

In [29]:
df_merged.count()

appname                  6042
fancyname                6042
company                  6042
purchases                5995
ads                      5995
sharesinfo               5995
shareslocation           5995
unrestrictedinternet     5995
usersinteract            5995
game                     6042
age_rating               6042
Parentalguidance         6042
Downloads                5995
category                 6042
low_price_item           4592
top_price_item           4592
price                    6042
paidapp                  6042
rating                   6042
numberreviews            6042
cleaned_numberReviews    6042
App Name                 6042
Category                 6042
Rating                   6042
Rating Count             6042
Installs                 6042
Minimum Installs         6042
Maximum Installs         6042
Free                     6042
Price                    6042
Currency                 6042
Size                     6042
Minimum Android          6041
Developer 

In [30]:
# Drop N/A records

df_merged = df_merged[df_merged["Released"].notna()]

In [31]:
# UDF to calculate the difference between today's date and the given release date, expressed in Days

def CountDaysSinceRelease(releaseDate):
    return abs((date.today() - datetime.strptime(releaseDate, "%b %d, %Y").date()).days)

In [32]:
# Derive the daysOnMarket column with the above UDF for each row

df_merged['daysOnMarket'] = df_merged.apply(lambda row: CountDaysSinceRelease(row['Released']), axis=1)

C:\Users\artem\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
# Preview the data, complete with the daysOnMarket column

df_merged

,appname,fancyname,company,purchases,ads,sharesinfo,shareslocation,unrestrictedinternet,usersinteract,game,...,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,CompanyIsExperienced,daysOnMarket
0,com.gamebasics.osm,Online Soccer Manager (OSM),gamebasics bv,1.0,1.0,0.0,0.0,0.0,1.0,1,...,en.support@onlinesoccermanager.com,"Oct 4, 2012","Nov 26, 2020",Everyone,http://www.onlinesoccermanager.com/Privacy,True,True,False,False,3104
1,com.fgol.HungrySharkEvolution,Hungry Shark Evolution,ubisoft entertainment,1.0,1.0,0.0,0.0,0.0,0.0,1,...,android.support@ubisoft.com,"Feb 22, 2013","Oct 27, 2020",Teen,http://r-mob.ubi.com/?a=PrivacyPolicyUS,True,True,True,True,2963
2,com.zynga.livepoker,Zynga Poker ? Free Texas Holdem Online Card Games,zynga,1.0,1.0,1.0,0.0,0.0,1.0,1,...,android.support@zynga.com,"Dec 21, 2010","Nov 04, 2020",Teen,https://www.zynga.com/privacy/policy,True,True,False,True,3757
3,ee.dustland.android.dustlandsudoku,Sudoku - The Clean One,dustland design,1.0,1.0,0.0,0.0,0.0,0.0,1,...,sudoku@dustland.ee,"Jul 20, 2017","Oct 29, 2020",Everyone,http://dustland.ee/sudoku/privacy-policy/,True,True,False,False,1354
4,com.miniclip.eightballpool,8 Ball Pool,miniclip.com,1.0,1.0,0.0,0.0,0.0,0.0,1,...,support@miniclip.com,"Jan 23, 2013","Nov 26, 2020",Everyone,http://www.miniclip.com/android/privacy-policy/,True,True,True,True,2993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6037,com.berniiiiiiii.logomatchup,"""Memory"" - Memory game",berni mobile,NaN,NaN,NaN,NaN,NaN,NaN,1,...,girblumk@gmail.com,"Oct 26, 2012","Nov 28, 2020",Everyone,http://www.bernimobile.com/PrivatePolicy,True,False,False,True,3082
6038,com.gipnetix.berryking,Berry King,gameborn inc.,NaN,NaN,NaN,NaN,NaN,NaN,1,...,support@gameborn.io,"Jul 11, 2014","Aug 20, 2020",Everyone,https://gameborn.io/privacy-policy,True,True,False,False,2459
6039,com.smilerlee.jewels,Jewels Switch,words mobile,NaN,NaN,NaN,NaN,NaN,NaN,1,...,contact@words-mobile.com,"Jun 13, 2013","Dec 26, 2019",Everyone,http://www2.words-mobile.com/policy.htm,True,True,False,True,2852
6040,com.geargames.pfp,Prehistoric Park Builder,gear games,NaN,NaN,NaN,NaN,NaN,NaN,1,...,info@gear-games.com,"Nov 28, 2012","Feb 17, 2014",Everyone,http://gear-games.com/privacy/,True,True,False,True,3049


In [34]:
df_merged.columns

Index(['appname', 'fancyname', 'company', 'purchases', 'ads', 'sharesinfo',
       'shareslocation', 'unrestrictedinternet', 'usersinteract', 'game',
       'age_rating', 'Parentalguidance', 'Downloads', 'category',
       'low_price_item', 'top_price_item', 'price', 'paidapp', 'rating',
       'numberreviews', 'cleaned_numberReviews', 'App Name', 'Category',
       'Rating', 'Rating Count', 'Installs', 'Minimum Installs',
       'Maximum Installs', 'Free', 'Price', 'Currency', 'Size',
       'Minimum Android', 'Developer Id', 'Developer Website',
       'Developer Email', 'Released', 'Last Updated', 'Content Rating',
       'Privacy Policy', 'Ad Supported', 'In App Purchases', 'Editors Choice',
       'CompanyIsExperienced', 'daysOnMarket'],
      dtype='object')

In [35]:
# Derive the Release Year and Counter columns (used for calculation of average downloads per year)

df_merged["Release_Year"] = df_merged.apply (lambda row: row["Released"][-4:], axis=1)
df_merged["Counter"] = 1


C:\Users\artem\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\artem\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
df_merged

,appname,fancyname,company,purchases,ads,sharesinfo,shareslocation,unrestrictedinternet,usersinteract,game,...,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,CompanyIsExperienced,daysOnMarket,Release_Year,Counter
0,com.gamebasics.osm,Online Soccer Manager (OSM),gamebasics bv,1.0,1.0,0.0,0.0,0.0,1.0,1,...,"Nov 26, 2020",Everyone,http://www.onlinesoccermanager.com/Privacy,True,True,False,False,3104,2012,1
1,com.fgol.HungrySharkEvolution,Hungry Shark Evolution,ubisoft entertainment,1.0,1.0,0.0,0.0,0.0,0.0,1,...,"Oct 27, 2020",Teen,http://r-mob.ubi.com/?a=PrivacyPolicyUS,True,True,True,True,2963,2013,1
2,com.zynga.livepoker,Zynga Poker ? Free Texas Holdem Online Card Games,zynga,1.0,1.0,1.0,0.0,0.0,1.0,1,...,"Nov 04, 2020",Teen,https://www.zynga.com/privacy/policy,True,True,False,True,3757,2010,1
3,ee.dustland.android.dustlandsudoku,Sudoku - The Clean One,dustland design,1.0,1.0,0.0,0.0,0.0,0.0,1,...,"Oct 29, 2020",Everyone,http://dustland.ee/sudoku/privacy-policy/,True,True,False,False,1354,2017,1
4,com.miniclip.eightballpool,8 Ball Pool,miniclip.com,1.0,1.0,0.0,0.0,0.0,0.0,1,...,"Nov 26, 2020",Everyone,http://www.miniclip.com/android/privacy-policy/,True,True,True,True,2993,2013,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6037,com.berniiiiiiii.logomatchup,"""Memory"" - Memory game",berni mobile,NaN,NaN,NaN,NaN,NaN,NaN,1,...,"Nov 28, 2020",Everyone,http://www.bernimobile.com/PrivatePolicy,True,False,False,True,3082,2012,1
6038,com.gipnetix.berryking,Berry King,gameborn inc.,NaN,NaN,NaN,NaN,NaN,NaN,1,...,"Aug 20, 2020",Everyone,https://gameborn.io/privacy-policy,True,True,False,False,2459,2014,1
6039,com.smilerlee.jewels,Jewels Switch,words mobile,NaN,NaN,NaN,NaN,NaN,NaN,1,...,"Dec 26, 2019",Everyone,http://www2.words-mobile.com/policy.htm,True,True,False,True,2852,2013,1
6040,com.geargames.pfp,Prehistoric Park Builder,gear games,NaN,NaN,NaN,NaN,NaN,NaN,1,...,"Feb 17, 2014",Everyone,http://gear-games.com/privacy/,True,True,False,True,3049,2012,1


In [37]:
# Generate the downloadsByYear dataframe, containing grouping of game data by Release Year
# Average downloads is calculated by dividing the sum of Installations by the sum of games released for each Year

df_downloadsByYear = df_merged.groupby("Release_Year")[["Maximum Installs", "Counter"]].sum().reset_index()
df_downloadsByYear["Average_Downloads"] = df_downloadsByYear["Maximum Installs"] / df_downloadsByYear["Counter"]
df_downloadsByYear

,Release_Year,Maximum Installs,Counter,Average_Downloads
0,2010,1176046612,42,2.800111e+07
1,2011,3054603914,114,2.679477e+07
2,2012,9535835285,251,3.799138e+07
3,2013,13064386565,362,3.608947e+07
4,2014,11511702494,573,2.009023e+07
5,2015,11514870092,716,1.608222e+07
6,2016,16168517166,1013,1.596102e+07
7,2017,15470589630,1337,1.157112e+07
8,2018,14244825242,1195,1.192036e+07
9,2019,5685103561,418,1.360073e+07


In [38]:
# UDF for fetching the Average Downloads from the downloadsByYear dataframe for a given Year

def GetAverageDownloadsByYear(year):
    return df_downloadsByYear.loc[df_downloadsByYear["Release_Year"] == year].iloc[0]['Average_Downloads']
    

In [39]:
# Create the AverageDownloadsForYear column (incorporated directly into the merged dataframe, for simplicity of algorithm)
# Create the Relative Downloads column as a function of the Installations for each app record and the average downloads for the game's release year. This column is aimed to be a measure of how well a game performs compared to other games for the same year.

df_merged["AverageDownloadsForYear"] = df_merged.apply(lambda row: GetAverageDownloadsByYear(row["Release_Year"]), axis=1)
df_merged["RelativeDownloads"] = df_merged["Maximum Installs"] / df_merged["AverageDownloadsForYear"]
df_merged

C:\Users\artem\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\artem\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,appname,fancyname,company,purchases,ads,sharesinfo,shareslocation,unrestrictedinternet,usersinteract,game,...,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,CompanyIsExperienced,daysOnMarket,Release_Year,Counter,AverageDownloadsForYear,RelativeDownloads
0,com.gamebasics.osm,Online Soccer Manager (OSM),gamebasics bv,1.0,1.0,0.0,0.0,0.0,1.0,1,...,http://www.onlinesoccermanager.com/Privacy,True,True,False,False,3104,2012,1,3.799138e+07,1.110371
1,com.fgol.HungrySharkEvolution,Hungry Shark Evolution,ubisoft entertainment,1.0,1.0,0.0,0.0,0.0,0.0,1,...,http://r-mob.ubi.com/?a=PrivacyPolicyUS,True,True,True,True,2963,2013,1,3.608947e+07,9.903636
2,com.zynga.livepoker,Zynga Poker ? Free Texas Holdem Online Card Games,zynga,1.0,1.0,1.0,0.0,0.0,1.0,1,...,https://www.zynga.com/privacy/policy,True,True,False,True,3757,2010,1,2.800111e+07,2.870604
3,ee.dustland.android.dustlandsudoku,Sudoku - The Clean One,dustland design,1.0,1.0,0.0,0.0,0.0,0.0,1,...,http://dustland.ee/sudoku/privacy-policy/,True,True,False,False,1354,2017,1,1.157112e+07,0.221404
4,com.miniclip.eightballpool,8 Ball Pool,miniclip.com,1.0,1.0,0.0,0.0,0.0,0.0,1,...,http://www.miniclip.com/android/privacy-policy/,True,True,True,True,2993,2013,1,3.608947e+07,18.784716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6037,com.berniiiiiiii.logomatchup,"""Memory"" - Memory game",berni mobile,NaN,NaN,NaN,NaN,NaN,NaN,1,...,http://www.bernimobile.com/PrivatePolicy,True,False,False,True,3082,2012,1,3.799138e+07,0.319987
6038,com.gipnetix.berryking,Berry King,gameborn inc.,NaN,NaN,NaN,NaN,NaN,NaN,1,...,https://gameborn.io/privacy-policy,True,True,False,False,2459,2014,1,2.009023e+07,0.029217
6039,com.smilerlee.jewels,Jewels Switch,words mobile,NaN,NaN,NaN,NaN,NaN,NaN,1,...,http://www2.words-mobile.com/policy.htm,True,True,False,True,2852,2013,1,3.608947e+07,1.232902
6040,com.geargames.pfp,Prehistoric Park Builder,gear games,NaN,NaN,NaN,NaN,NaN,NaN,1,...,http://gear-games.com/privacy/,True,True,False,True,3049,2012,1,3.799138e+07,0.155639


In [42]:
# Review the completed merged dataframe columns, datatypes

pd.set_option('max_columns', None)
df_merged

,appname,fancyname,company,purchases,ads,sharesinfo,shareslocation,unrestrictedinternet,usersinteract,game,age_rating,Parentalguidance,Downloads,category,low_price_item,top_price_item,price,paidapp,rating,numberreviews,cleaned_numberReviews,App Name,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,Currency,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,CompanyIsExperienced,daysOnMarket,Release_Year,Counter,AverageDownloadsForYear,RelativeDownloads
0,com.gamebasics.osm,Online Soccer Manager (OSM),gamebasics bv,1.0,1.0,0.0,0.0,0.0,1.0,1,everyone,0.0,10000000.0,sports,0.59,109.99,free,0.0,4.5,"1,656,120",1656120,Online Soccer Manager (OSM) - 20/21,Sports,4.5,1903171.0,"10,000,000+",10000000.0,42184514,True,0.00,USD,28M,4.4 and up,Gamebasics BV,http://www.onlinesoccermanager.com,en.support@onlinesoccermanager.com,"Oct 4, 2012","Nov 26, 2020",Everyone,http://www.onlinesoccermanager.com/Privacy,True,True,False,False,3104,2012,1,3.799138e+07,1.110371
1,com.fgol.HungrySharkEvolution,Hungry Shark Evolution,ubisoft entertainment,1.0,1.0,0.0,0.0,0.0,0.0,1,16+,0.0,100000000.0,arcade,2.29,109.99,free,0.0,4.5,"6,495,234",6495234,Hungry Shark Evolution,Arcade,4.4,7010986.0,"100,000,000+",100000000.0,357416919,True,0.00,USD,Varies with device,4.1 and up,Ubisoft Entertainment,https://www.ubisoft.com/,android.support@ubisoft.com,"Feb 22, 2013","Oct 27, 2020",Teen,http://r-mob.ubi.com/?a=PrivacyPolicyUS,True,True,True,True,2963,2013,1,3.608947e+07,9.903636
2,com.zynga.livepoker,Zynga Poker ? Free Texas Holdem Online Card Games,zynga,1.0,1.0,1.0,0.0,0.0,1.0,1,12+,0.0,50000000.0,casino,0.59,350.00,free,0.0,4.5,"2,315,552",2315552,Zynga Poker – Free Texas Holdem Online Card Games,Casino,4.3,2527726.0,"50,000,000+",50000000.0,80380111,True,0.00,USD,98M,4.1 and up,Zynga,https://www.zynga.com/,android.support@zynga.com,"Dec 21, 2010","Nov 04, 2020",Teen,https://www.zynga.com/privacy/policy,True,True,False,True,3757,2010,1,2.800111e+07,2.870604
3,ee.dustland.android.dustlandsudoku,Sudoku - The Clean One,dustland design,1.0,1.0,0.0,0.0,0.0,0.0,1,everyone,0.0,1000000.0,puzzle,1.89,2.99,free,0.0,4.6,"6,639",6639,Sudoku - The Clean One,Puzzle,4.5,16316.0,"1,000,000+",1000000.0,2561887,True,0.00,USD,6.2M,4.4 and up,Dustland Design,http://dustland.ee,sudoku@dustland.ee,"Jul 20, 2017","Oct 29, 2020",Everyone,http://dustland.ee/sudoku/privacy-policy/,True,True,False,False,1354,2017,1,1.157112e+07,0.221404
4,com.miniclip.eightballpool,8 Ball Pool,miniclip.com,1.0,1.0,0.0,0.0,0.0,0.0,1,everyone,0.0,500000000.0,sports,0.79,99.99,free,0.0,4.5,"17,053,699",17053699,8 Ball Pool,Sports,4.4,20048123.0,"500,000,000+",500000000.0,677930380,True,0.00,USD,66M,4.4 and up,Miniclip.com,https://support.miniclip.com/hc/categories/115...,support@miniclip.com,"Jan 23, 2013","Nov 26, 2020",Everyone,http://www.miniclip.com/android/privacy-policy/,True,True,True,True,2993,2013,1,3.608947e+07,18.784716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6037,com.berniiiiiiii.logomatchup,"""Memory"" - Memory game",berni mobile,NaN,NaN,NaN,NaN,NaN,NaN,1,everyone,0.0,NaN,puzzle,NaN,NaN,free,0.0,4.1,84.91k+,84910,"""Memory"" - Memory game",Puzzle,4.6,88138.0,"10,000,000+",10000000.0,12156758,True,0.00,USD,3.1M,4.1 and up,Berni Mobile,http://www.bernimobile.com/,girblumk@gmail.com,"Oct 26, 2012","Nov 28, 2020",Everyone,http://www.bernimobile.com/PrivatePolicy,True,False,False,True,3082,2012,1,3.799138e+07,0.319987
6038,com.gipnetix.berryking,Berry King,gameborn inc.,NaN,NaN,NaN,NaN,NaN,NaN,1,everyone,0.0,NaN,puzzle,NaN,NaN,free,0.0,4.4,35.66k+,35660,Berry King,Puzzle,4.6,35553.0,"500,000+",500000.0,586977,True,0.00,USD,23M,7.0 and up,GAMEBORN Inc.,https://gameborn.io,support@gameborn.io,"Jul 1

In [43]:
df_merged.dtypes

appname                     object
fancyname                   object
company                     object
purchases                  float64
ads                        float64
sharesinfo                 float64
shareslocation             float64
unrestrictedinternet       float64
usersinteract              float64
game                         int64
age_rating                  object
Parentalguidance           float64
Downloads                  float64
category                    object
low_price_item              object
top_price_item              object
price                       object
paidapp                    float64
rating                     float64
numberreviews               object
cleaned_numberReviews        int64
App Name                    object
Category                    object
Rating                     float64
Rating Count               float64
Installs                    object
Minimum Installs           float64
Maximum Installs             int64
Free                

In [44]:
# Create enhanced Companies DF (cleaned up for loading into SQL server)
df_companies_enhanced = df_merged.groupby("Developer Id")["Developer Id"].count().reset_index(name="GamesCount").rename(columns={'Developer Id':'CompanyName'})
df_companies_enhanced

,CompanyName,GamesCount
0,(Andrei & Aleksandr Krupiankou),1
1,(주)엔비젼크리에이티브,1
2,0km apps,1
3,100%Game,2
4,100500games.org,1
...,...,...
3128,головоломка world studio,2
3129,✨ Bizo Mobile,1
3130,ふんどしパレード,1
3131,ザイザックス株式会社,1


In [45]:
# Preview the enhanced companies dataframe

df_companies

,company,gamesCount
0,(<u+c8fc>)<u+c5d4><u+be44><u+c83c><u+d06c><u+b...,1
1,(andrei & aleksandr krupiankou),1
2,0.1%,2
3,0km apps,1
4,100500games.org,1
...,...,...
3142,zut!,1
3143,zuuks games,3
3144,zy puzzle games,1
3145,zynga,19


In [46]:
# Export clean csv for machine learning purposes
# Legacy code - data is now staged in SQL server

#df_merged.to_csv("allcleaned.csv", index=False)
#df_downloadsByYear.to_csv("downloadsByYear.csv", index=False)

PermissionError: [Errno 13] Permission denied: 'allcleaned.csv'

In [86]:
# DB connection for staging DB
import pyodbc

server = 'localhost' 
database = 'AnalyticsBootcampDB' 
username = 'bootcamp' 
password = 'bootcamp' 

conString = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

conn = pyodbc.connect(conString)
cursor = conn.cursor()

In [ ]:
# Cleanup functions to convert data to a format appropriate for the datatype of destination columns for bulk insertion

In [87]:
def CleanStringField(inputVal):
    cleanedString = str(inputVal) \
    .replace("(","") \
    .replace(")","") \
    .replace("'","''") \
    .replace(",","")
    
    return "'"+cleanedString+"'"

In [50]:
def CleanBitField(inputVal):
    if(inputVal == 1 or inputVal == "TRUE" or inputVal == "true" or inputVal == "True"):
        return "1"
    elif(inputVal == 0 or inputVal == "FALSE" or inputVal == "false" or inputVal == "False"):
        return "0"
    else:
        return "NULL"

In [71]:
def CleanNumberField(inputVal):
    if(inputVal is None or str(inputVal) == "nan"):
        return "NULL"
    else:
        return str(inputVal)

In [88]:
# Write company data to SQL Server

for index, row in df_companies.iterrows():
    companyName = CleanStringField(row.company)
    
    statement = f"INSERT INTO dbo.CompaniesStage (CompanyName,GamesCount) VALUES ({companyName},'{row.gamesCount}')"
    #print(statement)
    cursor.execute(statement)
    
conn.commit()

In [84]:
# Write app data to SQL Server
appsStageColumns="[AppName],[AppFancyName],[CompanyName],[Purchases],[Ads],[SharesInfo],[SharesLocation],[UnrestrictedInternet],[UsersInteract],[AgeRating],[ParentalGuidance],[Downloads],[Category],[LowPrice],[TopPrice],[Price],[PaidApp],[Rating],[RatingCount],[Installs],[MinimumInstalls],[MaximumInstalls],[Free],[ReleaseDate],[LastUpdated],[EditorsChoice],[DaysOnMarket],[ReleaseYear],[RelativeDownloadsPerformance]"

for index, row in df_merged.iterrows():
    appName = CleanStringField(row['appname'])
    appFancyName = CleanStringField(row['fancyname'])
    companyName = CleanStringField(row['company'])
    purchases = CleanBitField(row['purchases'])
    ads = CleanBitField(row['ads'])
    sharesInfo = CleanBitField(row['sharesinfo'])
    sharesLocation = CleanBitField(row['shareslocation'])
    unrestrictedInternet = CleanBitField(row['unrestrictedinternet'])
    usersInteract = CleanBitField(row['usersinteract'])
    ageRating = CleanStringField(row['Content Rating'])
    parentalGuidance = CleanBitField(row['Parentalguidance'])
    downloads = CleanNumberField(row['Downloads'])
    category = CleanStringField(row['category'])
    lowPrice = CleanNumberField(row['low_price_item'])
    topPrice = CleanNumberField(row['top_price_item'])
    price = CleanStringField(row['price'])
    paidApp = CleanBitField(row['paidapp'])
    rating = row['rating']
    ratingCount = row['cleaned_numberReviews']
    installs = CleanStringField(row['Installs'])
    minimumInstalls = row['Minimum Installs']
    maximumInstalls = row['Maximum Installs']
    free = CleanBitField(row['Free'])
    releaseDate = CleanStringField(row['Released'])
    lastUpdated = CleanStringField(row['Last Updated'])
    editorsChoice = CleanBitField(row['Editors Choice'])
    daysOnMarket = row['daysOnMarket']
    releaseYear = CleanStringField(row['Release_Year'])
    relativeDownloadsPerformance = row['RelativeDownloads']
    
    statement = f"INSERT INTO dbo.AppsStage ({appsStageColumns}) VALUES ({appName},{appFancyName},{companyName},{purchases},{ads},{sharesInfo},{sharesLocation},{unrestrictedInternet},{usersInteract},{ageRating},{parentalGuidance},{downloads},{category},{lowPrice},{topPrice},{price},{paidApp},{rating},{ratingCount},{installs},{minimumInstalls},{maximumInstalls},{free},{releaseDate},{lastUpdated},{editorsChoice},{daysOnMarket},{releaseYear},{relativeDownloadsPerformance})"
    cursor.execute(statement)
    #print(statement)
conn.commit()
    

In [85]:
# Release the connection
conn.close()